In [1]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
df_movie = pd.read_pickle('./Data/api.pickle')

In [3]:
df_movie.head()

,belongs_to_collection,budget,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,actors,directors,release_day,release_month
0,None,0.0,fi,3.860491,"[Villealfa Filmproduction Oy, Finnish Film Fou...",[Finland],1988-10-21,0.0,69.0,Ariel,7.1,44.0,1988,0.000000,"[Turo Pajala, Susanna Haavisto, Matti Pellonpä...",[Aki Kaurismäki],Friday,10
1,None,0.0,fi,2.292110,[Villealfa Filmproduction Oy],[Finland],1986-10-16,0.0,76.0,Shadows in Paradise,7.1,35.0,1986,0.000000,"[Matti Pellonpää, Kati Outinen, Sakari Kuosman...",[Aki Kaurismäki],Thursday,10
2,None,4000000.0,en,9.026586,"[Miramax Films, A Band Apart]",[United States of America],1995-12-09,4300000.0,98.0,Four Rooms,6.5,539.0,1995,1.075000,"[Tim Roth, Antonio Banderas, Jennifer Beals, M...","[Allison Anders, Alexandre Rockwell, Robert Ro...",Saturday,12
3,None,0.0,en,5.538671,"[Universal Pictures, Largo Entertainment, JVC ...","[Japan, United States of America]",1993-10-15,12136938.0,110.0,Judgment Night,6.4,79.0,1993,0.000000,"[Emilio Estevez, Cuba Gooding Jr., Denis Leary...",[Stephen Hopkins],Friday,10
4,Star Wars Collection,11000000.0,en,42.149697,"[Lucasfilm, Twentieth Century Fox Film Corpora...",[United States of America],1977-05-25,775398007.0,121.0,Star Wars,8.1,6778.0,1977,70.490728,"[Mark Hamill, Harrison Ford, Carrie Fisher, Pe...",[George Lucas],Wednesday,5


In [6]:
df_movie.dtypes

belongs_to_collection            object
budget                          float64
genres                           object
original_language                object
overview                         object
popularity                      float64
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
tagline                          object
title                            object
vote_average                    float64
vote_count                      float64
release_year                      int64
return                          float64
actors                           object
directors                        object
dtype: object

In [7]:
df_movie.isna().sum()

belongs_to_collection    40534
budget                       0
genres                    2327
original_language           10
overview                   907
popularity                   0
production_companies     11579
production_countries      6089
release_date                 0
revenue                      0
runtime                    240
spoken_languages          3702
tagline                  24695
title                        0
vote_average                 0
vote_count                   0
release_year                 0
return                       0
actors                    2303
directors                  698
dtype: int64

In [23]:
# Esta funcion pide como parametro un nombre corto de un idioma y retorna un diccionario con el idioma ingresado y la cantidad de peliculas en ese idioma
def peliculas_idioma(idioma:str):
    if not isinstance(idioma, str):
        return 'Debe ingresar un texto'
    else:
        respuesta = df_movie[df_movie['original_language'] == idioma].shape[0]

    return {'idioma':idioma, 'cantidad': str(respuesta)}

print(peliculas_idioma('en'))

{'idioma': 'en', 'cantidad': '31938'}


In [21]:
# esta funcion pide como parametro en nombre de una pelicula y retorna un diccionario con en nombre de la pelicula, duracion en minutos y el año de estreno
def peliculas_duracion(pelicula:str):
    if not isinstance(pelicula, str):
        return 'Debe ingresar un texto'
    elif not (df_movie['title'] == pelicula).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    else:
        df = df_movie[df_movie['title'] == pelicula][['runtime','release_year']]
        respuesta = df['runtime'][0]
        anio = df['release_year'][0]
        return {'pelicula':pelicula, 'duracion':str(respuesta), 'anio':str(anio)}

print(peliculas_duracion('Ariel'))

{'pelicula': 'Ariel', 'duracion': '69.0', 'anio': '1988'}


In [10]:
# Esta funcion pide como parametro una franquicia y retorna un diccionario con el nombre de la franquicia, cantidad de peliculas que tiene la franquicia, ganancia total (return), y granancia promedio.
def franquicia(franquicia:str):
    if not isinstance(franquicia, str):
        return 'Debe ingresar un texto'
    elif not (df_movie['belongs_to_collection'] == franquicia).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    else:
        df = df_movie[df_movie['belongs_to_collection'] == franquicia]
        cantidad = df.shape[0]
        ganancia_total = df['return'].sum()
        promedio = ganancia_total/cantidad

        return {'franquicia':franquicia, 'cantidad':str(cantidad), 'ganancia_total': str(ganancia_total), 'ganancia_promedio':str(promedio)}

print(franquicia('Star Wars Collection'))

{'franquicia': 'Star Wars Collection', 'cantidad': '8', 'ganancia_total': '152.79842082211397', 'ganancia_promedio': '19.099802602764246'}


In [24]:
# Esta funcion pide como parametro un nombre de pais y retorna un diccionario con el nombre del pais y la cantidad de peliculas que se filmaron en ese pais.
def peliculas_pais(pais:str):
    if not isinstance(pais, str):
        return 'Debe ingresar un texto'
    else:
        cantidad = df_movie[df_movie['production_countries'].apply(lambda paises: pais in paises if isinstance(paises, list) else False)].shape[0]
        return {'pais':pais, 'cantidad': str(cantidad)}

print(peliculas_pais('United States of America'))

{'pais': 'United States of America', 'cantidad': '21044'}


In [25]:
# Esta funcion pide como parametro el nombre de una productora y devuelve un diccionario con el nombre de la productora, recaudacion total (revenue) y cantidad de peliculas
def productoras_exitosas(productora:str):
    if not isinstance(productora, str):
        return 'Debe ingresar un texto'
    else:
        temp_df = df_movie[df_movie['production_companies'].apply(lambda companies: productora in companies if isinstance(companies, list) else False)]
        cantidad = temp_df.shape[0]
        revenue_total = temp_df['revenue'].sum()
        return {'productora':productora, 'revenue_total': str(revenue_total),'cantidad':str(cantidad)}

print(productoras_exitosas('Lucasfilm'))

{'productora': 'Lucasfilm', 'revenue_total': '9898421106.0', 'cantidad': '29'}


***

In [132]:
def cantidad_filmaciones_mes(mes:str):
    months = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12
    }

    if not isinstance(mes, str):
        return 'Debe ingresar un texto, por ejemplo, por ejemplo: enero'
    elif mes.lower() not in months:
        return 'Debe ingresar un mes válido del año, por ejemplo: enero'
    else:
        mes = mes.lower()
        month = months[mes]
        resp = df_movie[(df_movie['release_month'] == month)]['release_month'].count()
        return {'mes': mes, 'cantidad': str(resp)}

In [133]:
months = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12,
        "sabado":0,
        12:0
    }
for a in months:
    print(cantidad_filmaciones_mes(a))

{'mes': 'enero', 'cantidad': '5831'}
{'mes': 'febrero', 'cantidad': '3006'}
{'mes': 'marzo', 'cantidad': '3532'}
{'mes': 'abril', 'cantidad': '3423'}
{'mes': 'mayo', 'cantidad': '3325'}
{'mes': 'junio', 'cantidad': '3131'}
{'mes': 'julio', 'cantidad': '2630'}
{'mes': 'agosto', 'cantidad': '3357'}
{'mes': 'septiembre', 'cantidad': '4798'}
{'mes': 'octubre', 'cantidad': '4581'}
{'mes': 'noviembre', 'cantidad': '3629'}
{'mes': 'diciembre', 'cantidad': '3766'}
Debe ingresar un mes válido del año, por ejemplo: enero
Debe ingresar un texto, por ejemplo, por ejemplo: enero


In [98]:
def cantidad_filmaciones_dia(dia:str):
    days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miércoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sábado': 'Saturday',
    'domingo': 'Sunday'}

    if not isinstance(dia, str):
        return 'Debe ingresar un texto, por ejemplo: sabado'
    elif dia.lower() not in days:
        return 'Debe ingresar un dia de la semana, por ejemplo: sabado'
    else:
        dia = dia.lower()
        day = days[dia]
        resp = df_movie[(df_movie['release_day'] == day)]['release_day'].count()
        return {'dia':dia, 'cantidad':resp}

In [99]:
days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miércoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sábado': 'Saturday',
    'domingo': 'Sunday',
    "enero": 0,
    12:0}
for a in days:
    print(cantidad_filmaciones_dia(a))

{'dia': 'lunes', 'cantidad': 3474}
{'dia': 'martes', 'cantidad': 4598}
{'dia': 'miércoles', 'cantidad': 6986}
{'dia': 'jueves', 'cantidad': 7466}
{'dia': 'viernes', 'cantidad': 13783}
{'dia': 'sábado', 'cantidad': 5112}
{'dia': 'domingo', 'cantidad': 3590}
Debe ingresar un dia de la semana, por ejemplo: sabado
Debe ingresar un texto, por ejemplo: sabado


In [129]:
def score_titulo(titulo:str):
    if not isinstance(titulo, str):
        return 'Debe ingresar un texto'
    elif not (df_movie == titulo).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    else:
        resp = df_movie[df_movie['title'] == titulo][['title','popularity','release_year']].to_dict(orient='records')[0]
        return {'titulo':resp['title'], 'popularidad':resp['popularity'], 'año':resp['release_year']}

In [130]:
print(score_titulo('123'))
print(score_titulo('jumanji'))
print(score_titulo(6534))

La pelicula no se encuentra en la base de datos
La pelicula no se encuentra en la base de datos
Debe ingresar un texto


In [102]:
def votos_titulo(titulo:str):
    if not isinstance(titulo, str):
        return 'Debe ingresar un texto'
    elif not (df_movie == titulo).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    elif int(df_movie[df_movie['title'] == titulo]['vote_count']) > 2000:
        resp = df_movie[df_movie['title'] == titulo][['title','vote_count','vote_average']].to_dict(orient='records')[0]
        return {'titulo':resp['title'], 'voto_total':resp['vote_count'], 'voto_promedio':resp['vote_average']}
    else:
        return 'La consulta no cumple con las condicion por ende no se devuelve informacion'

In [103]:
votos_titulo('sgw54')

'La pelicula no se encuentra en la base de datos'

In [112]:
def get_actor(nombre_actor:str):
    if not isinstance(nombre_actor, str):
        return 'Debe ingresar un texto'
    
    elif not any(nombre_actor in sublist for sublist in df_movie['actors'] if sublist):
        return 'El actor no se encuentra en la base de datos'

    _return = 0
    n_films = 0
    avg = 0
    for index,lista in enumerate(df_movie['actors']):
        if lista: 
            if nombre_actor in lista:
                n_films += 1
                _return += df_movie['return'].iloc[index]
    
    if _return == 0 and n_films == 0:
        avg = 0
    else:
        avg = _return/n_films

    return {'actor':nombre_actor, 'cantidad_filmaciones':n_films, 'retorno_total':_return, 'retorno_promedio':avg}

In [113]:
get_actor('sdfsd')

'El actor no se encuentra en la base de datos'

In [125]:
def get_director(nombre_director:str):
    if not isinstance(nombre_director, str):
        return 'Debe ingresar un texto'
    elif not any(nombre_director in sublist for sublist in df_movie['directors'] if sublist):
        return 'El director no se encuentra en la base de datos'

    total_return = 0
    movies = []
    years = []
    _return = []
    budget = []
    revenue = []
    for index,lista in enumerate(df_movie['directors']):
        if lista:
            if nombre_director in lista:
                total_return += df_movie['return'][index]
                movies.append(df_movie['title'][index])
                years.append(df_movie['release_year'][index])
                _return.append(df_movie['return'][index])
                budget.append(df_movie['budget'][index])
                revenue.append(df_movie['revenue'][index])

    return {'director':nombre_director, 'retorno_total_director':total_return, 
    'peliculas':movies, 'año':years, 'retorno_pelicula':_return, 
    'budget_pelicula':budget, 'revenue_pelicula':revenue}

In [126]:
get_director('Aki Kaurismäki')

{'director': 'Aki Kaurismäki',
 'retorno_total_director': 3.079139631884058,
 'peliculas': ['Ariel',
  'Shadows in Paradise',
  'Lights in the Dusk',
  'The Man Without a Past',
  'The Match Factory Girl',
  'I Hired a Contract Killer',
  'Drifting Clouds',
  'Hamlet Goes Business',
  'Visions of Europe',
  'Leningrad Cowboys Go America',
  'Ten Minutes Older: The Trumpet',
  'Chloe',
  'An Unreasonable Man',
  'More',
  'Blood Out',
  'The Mask',
  "Saimir's decision",
  'Mr. & Mrs. Incredible',
  'Safe',
  'Hansel and Gretel',
  'Fire-Eater',
  'The Wait',
  'Running Wild',
  'Justin Timberlake + The Tennessee Kids'],
 'año': [1988,
  1986,
  2006,
  2002,
  1990,
  1990,
  1996,
  1987,
  2004,
  1989,
  2002,
  2009,
  2007,
  1998,
  2011,
  1961,
  2004,
  2011,
  2012,
  1988,
  1998,
  2013,
  1995,
  2016],
 'retorno_pelicula': [0.0,
  0.0,
  1.1703028985507247,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5639638666666666,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0

In [17]:
def recomendacion(titulo:str):
    if not isinstance(titulo, str):
        return 'Debe ingresar un texto'
    elif not (df_movie == titulo).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    # Cargo los modelos entrenados
    tfidf_model = joblib.load("./Models/tfidf_model.pkl")
    knn_model = joblib.load("./Models/knn_model.pkl")

    title_vector = tfidf_model.transform([titulo])  # Transforma el título de entrada en una representación vectorial
    _, indices = knn_model.kneighbors(title_vector)  # Busca los vecinos más cercanos basados en la representación vectorial
    recommendations = df_movie["title"].iloc[indices[0][1:]].tolist()  # Obtiene los títulos de las películas más similares (excluyendo la película de referencia)
    return {'lista recomendada': str(recommendations)}

In [18]:
recomendacion("Manuel on the Island of Wonders")

{'lista recomendada': "['Un marido de ida y vuelta', 'The Sandwich Man', 'Chernobyl Diaries', 'Friday the 13th: A New Beginning', 'Aftershock']"}